# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv("output_data/cities.csv")
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,5.87,60,100,2.68,CA,1633720500
1,Atuona,-9.8000,-139.0333,25.53,72,6,7.06,PF,1633720640
2,Mataura,-46.1927,168.8643,9.71,100,100,1.35,NZ,1633720640
3,Narsaq,60.9167,-46.0500,6.12,50,2,1.71,GL,1633720641
4,Bathsheba,13.2167,-59.5167,28.34,78,40,5.14,BB,1633720641


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Set up locations and weight for the map
locations = cities_data[['Lat', 'Lng']]
weights = cities_data['Humidity']

# Display heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities = cities_data.loc[(cities_data['Max Temp'] <= 24.4) & (cities_data['Humidity'] <= 68) & (cities_data['Wind Speed'] <= 2) & (cities_data['Cloudiness'] <= 0)]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Rzhaksa,52.1338,42.0264,4.71,57,0,1.81,RU,1633720644
105,Mordovo,52.0834,40.7700,5.16,55,0,1.97,RU,1633720687
136,Heraclea,37.5133,34.0467,14.19,31,0,0.84,TR,1633720701
155,Bodri,23.1667,81.4333,22.97,68,0,1.81,IN,1633720711
357,Marquard,-28.6645,27.4305,16.03,22,0,1.58,ZA,1633720800
425,Niğde,37.8333,34.7500,10.05,29,0,0.62,TR,1633720833
476,Korla,41.7597,86.1469,6.14,33,0,1.72,CN,1633720858
513,Milas,37.3164,27.7839,15.64,67,0,1.54,TR,1633720876
527,Tingi,-11.3000,35.0333,19.43,51,0,0.78,TZ,1633720883
558,Jurm,36.8648,70.8342,10.31,47,0,0.85,AF,1633720897


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create the hotels dataframe with a column for Hotel Name
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng']]
hotel_df

,City,Country,Lat,Lng
11,Rzhaksa,RU,52.1338,42.0264
105,Mordovo,RU,52.0834,40.7700
136,Heraclea,TR,37.5133,34.0467
155,Bodri,IN,23.1667,81.4333
357,Marquard,ZA,-28.6645,27.4305
425,Niğde,TR,37.8333,34.7500
476,Korla,CN,41.7597,86.1469
513,Milas,TR,37.3164,27.7839
527,Tingi,TZ,-11.3000,35.0333
558,Jurm,AF,36.8648,70.8342


In [7]:
# Set up the base url to query a nearby search in google API
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the cities dataframe, using the latitude and longitude data for the location parameter in the API request.
# Store the hotel name, latitude and longitude in lists. Use the latitude and longitude from the query to 
# perform another query for the hotel's city and country. Store each in separate lists
hotel_names = []
for index, row in hotel_df.iterrows():
    latitude = row[2]
    longitude = row[3]
    hotel_query_url = f"{nearby_url}?keyword=hotel&location={latitude}%2C{longitude}&radius=5000&type=hotel&key={g_key}"
    hotel_response = requests.get(hotel_query_url).json()
    try:
        hotel_names.append(hotel_response['results'][0]['name'])
    except:
        hotel_names.append('No hotel within 5000 meters!')

# Use the lists to fill in the hotels dataframe
hotel_df['Hotel Name'] = hotel_names
hotel_df

<ipython-input-7-5fac9a2bdaec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_names


,City,Country,Lat,Lng,Hotel Name
11,Rzhaksa,RU,52.1338,42.0264,Gostinitsa Uyut
105,Mordovo,RU,52.0834,40.7700,No hotel within 5000 meters!
136,Heraclea,TR,37.5133,34.0467,CHERRY GARDEN CITY&SPA HOTEL
155,Bodri,IN,23.1667,81.4333,No hotel within 5000 meters!
357,Marquard,ZA,-28.6645,27.4305,Le Cheri Country Guesthouse
425,Niğde,TR,37.8333,34.7500,No hotel within 5000 meters!
476,Korla,CN,41.7597,86.1469,The Garden Hotel Korla
513,Milas,TR,37.3164,27.7839,Milashan Hotel
527,Tingi,TZ,-11.3000,35.0333,No hotel within 5000 meters!
558,Jurm,AF,36.8648,70.8342,No hotel within 5000 meters!


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='black',
    stroke_color='purple', scale=5,
    info_box_content=hotel_info
)
# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))